<a href="https://colab.research.google.com/github/KhondamirRustamov/AF2seq/blob/main/AF2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title setup
%%time
import os
if not os.path.isdir("params"):
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar")
  os.system("tar -xf alphafold_params_2022-12-06.tar -C params")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from IPython.display import HTML
from google.colab import files
import numpy as np

def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

In [ ]:
import pickle
from google.colab import files
import os

pbd_name = '3vga.pdb' #@param {type:"string"}
chains = 'A' #@param {type:"string"}
hotspots = '1,2,3,4,5,6,7,8' #@param {type:"string"}
length = 20 #@param {type:"integer"}
number_of_seqs = 1 #@param {type:"integer"}

results_dic = 'results' #@param {type:"string"}

for i in range(number_of_seqs):
    print(f'Start {i}')
    clear_mem()
    af_model = mk_afdesign_model(protocol="binder", use_templates=True)
    af_model.prep_inputs(pdb_filename=pbd_name, chain=chains, binder_len=length, rm_aa='C', hotspot=hotspots)

    af_model.set_opt("con",num=2)
    af_model.set_opt("i_con",num=1)
    af_model.opt["weights"]["plddt"] = 0.1
    af_model.opt["weights"]["fape"] = 1.0

    af_model.design_3stage(100,100,10)

    #af_model.opt["weights"]["fape"] = 1.0
    #af_model.opt["weights"]["dgram_cce"] = 0.0

    #af_model.design_semigreedy(20, tries=20, models=[0,1], num_models=2)

    best_d = af_model._tmp['best']['aux']

    with open(f'{results_dic}/traj_{i}.pickle', 'wb') as handle:
        pickle.dump(best_d, handle, protocol=pickle.HIGHEST_PROTOCOL)

    af_model.save_pdb(f"{results_dic}/{af_model.protocol}_{i}.pdb")
    print(f'Finish {i}')



results_zip = f"{results_dic}.zip"
os.system(f"zip -r {results_zip} {results_dic}")
files.download(f"{results_zip}")

In [ ]:
#@title plot
af_model.plot_pdb()

In [ ]:
#@title animate
HTML(af_model.animate())

In [ ]:
#@title get sequences
af_model.get_seqs()

In [ ]:
#@title Package and download results.
from google.colab import files
import os

results_zip = f"{results_dic}.zip"
os.system(f"zip -r {results_zip} {results_dic}")
files.download(f"{results_zip}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>